In [1]:
from datasketch import MinHash, MinHashLSH
from multiprocessing import Pool, cpu_count
import pandas as pd
from tqdm import tqdm  # For progress tracking
import re

tqdm.pandas()

In [2]:
 cxr_reports = pd.read_csv("/scratch/baj321/cxr_reports.csv")
mimic_notes = pd.read_csv("/scratch/baj321/MIMIC-Note/physionet.org/files/mimic-iv-note/2.2/note/radiology.csv")

In [3]:
cxr_reports.columns

Index(['subject_id', 'study_id', 'cxr_report_text'], dtype='object')

In [4]:
print(len(cxr_reports.subject_id.unique()))
print(len(cxr_reports))

65379
227835


In [5]:
print(len(mimic_notes.subject_id.unique()))
print(len(mimic_notes))

237427
2321355


In [6]:
# Finding subject_ids in mimic_notes but not in cxr_reports
mimic_only_subject_ids = set(mimic_notes['subject_id'].unique()) - set(cxr_reports['subject_id'].unique())

# Finding subject_ids in cxr_reports but not in mimic_notes
cxr_only_subject_ids = set(cxr_reports['subject_id'].unique()) - set(mimic_notes['subject_id'].unique())

mimic_only_subject_ids = list(mimic_only_subject_ids)
cxr_only_subject_ids = list(cxr_only_subject_ids)

print("Number of subject_ids in mimic_notes but not in cxr_reports:", len(mimic_only_subject_ids))
print("Number of subject_ids in cxr_reports but not in mimic_notes:", len(cxr_only_subject_ids))


Number of subject_ids in mimic_notes but not in cxr_reports: 175710
Number of subject_ids in cxr_reports but not in mimic_notes: 3662


In [7]:

cxr_only_reports = cxr_reports[cxr_reports['subject_id'].isin(cxr_only_subject_ids)]

num_cxr_only_reports = len(cxr_only_reports)

print("Number of reports in cxr_reports for patients with no mimic notes:", num_cxr_only_reports)


Number of reports in cxr_reports for patients with no mimic notes: 12013


In [8]:
mimic_only_notes = mimic_notes[mimic_notes['subject_id'].isin(mimic_only_subject_ids)]

num_mimic_only_reports = len(mimic_only_notes)

print("Number of reports in mimic notes for patients with no cxr reports:", num_mimic_only_reports)

Number of reports in mimic notes for patients with no cxr reports: 1223439


In [9]:
def preprocess(text):
    if pd.isnull(text):
        print('text is', text)
        return set()
    
    # Step 1: Remove everything up to the literal string "FINAL REPORT\n"
    match = re.search(r"FINAL REPORT\n", text, re.IGNORECASE)
    if match:
        text = text[match.end():]  # Keep only text after "FINAL REPORT\n"
    text = re.sub(r"___M|___F|___|[.,!?/:;\\]", "", text)
    # Step 3: Remove all formatting: no spaces, literal "\n" strings, or newlines
    text = re.sub(r"\s+|\\n", "", text)  # Remove spaces and literal "\n" strings
    return text
    
def preprocess_mimic_notes(text):
    if pd.isnull(text):
        print('text is', text)
        return set()
        
    text = re.sub(r"___M|___F|___|[.,!?/:;\\]", "", text)
    
    # Step 2: Remove all formatting: no spaces, literal "\n" strings, or newlines
    text = re.sub(r"\s+|\\n", "", text)  # Remove spaces and literal "\n" strings
    return text

In [26]:
cxr_reports['processed_text'] = cxr_reports['cxr_report_text'].progress_apply(preprocess)

100%|██████████| 227835/227835 [00:14<00:00, 15262.13it/s]


In [11]:
mimic_only_notes['processed_text'] = mimic_only_notes['text'].progress_apply(preprocess_mimic_notes)

100%|██████████| 1223439/1223439 [02:26<00:00, 8373.65it/s]
/scratch/baj321/conda-envs/medfuse_py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
unique_mimic_notes = mimic_only_notes[~mimic_only_notes['processed_text'].isin(cxr_reports['processed_text'])]
len(unique_mimic_notes)
#185 notes whose processed text matches across both even though they have differing patient ID

1223254

In [15]:
repeated_text_mimic = mimic_only_notes[mimic_only_notes['processed_text'].isin(cxr_reports['processed_text'])]
len(repeated_text_mimic)

185

In [16]:
repeated_text_mimic.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,processed_text
8033,10033312-RR-72,10033312,NaN,RR,72,2156-01-18 13:11:00,2156-01-18 13:40:00,HISTORY: Preoperative.\n\nFINDINGS: In compa...,HISTORYPreoperativeFINDINGSIncomparisonwiththe...
13567,10061517-RR-24,10061517,NaN,RR,24,2147-07-15 15:32:00,2147-07-15 16:10:00,PA AND LATERAL VIEWS OF THE CHEST\n\nREASON FO...,PAANDLATERALVIEWSOFTHECHESTREASONFOREXAMCougha...
18622,10089210-RR-10,10089210,NaN,RR,10,2172-06-26 19:23:00,2172-06-26 19:54:00,EXAMINATION:\nChest: Frontal and lateral view...,EXAMINATIONChestFrontalandlateralviewsINDICATI...
27750,10127166-RR-42,10127166,NaN,RR,42,2152-06-04 11:47:00,2152-06-04 12:26:00,PA AND LATERAL VIEWS OF THE CHEST\n\nREASON FO...,PAANDLATERALVIEWSOFTHECHESTREASONFOREXAMCoughC...
41303,10186987-RR-67,10186987,NaN,RR,67,2190-10-30 09:45:00,2190-10-30 10:58:00,HISTORY: Positive PPD.\n\nFINDINGS: In compa...,HISTORYPositivePPDFINDINGSIncomparisonwithstud...


In [17]:
repeated_text_cxr = cxr_reports[cxr_reports['processed_text'].isin(mimic_only_notes['processed_text'])]
len(repeated_text_cxr)

284

In [18]:
repeated_text_cxr.head()

,subject_id,study_id,cxr_report_text,processed_text
253,14061701,58631426,FINAL REPORT\...,HISTORYPositivePPDFINDINGSNopreviousimagesNoev...
2592,14085712,53320221,FINAL REPORT\...,EXAMINATIONCHEST(PAANDLAT)INDICATIONHistorywit...
2637,14003716,58874157,FINAL REPORT\...,EXAMINATIONCHEST(PAANDLAT)INDICATIONwithchestp...
3501,14463099,54168089,FINAL REPORT\n,
3530,14738657,57614441,FINAL REPORT\...,EXAMINATIONChestFrontalandlateralviewsINDICATI...


In [21]:
# Filter cxr_reports where processed_text is empty
empty_processed_text_reports = cxr_reports[cxr_reports['processed_text'].isna() | (cxr_reports['processed_text'] == '')]

# Display or print the results
print("Number of reports with empty processed_text:", len(empty_processed_text_reports))
empty_processed_text_reports
#1 report only has the words FINAL REPORT

Number of reports with empty processed_text: 1


,subject_id,study_id,cxr_report_text,processed_text
3501,14463099,54168089,FINAL REPORT\n,


In [24]:
# Filter cxr_reports where processed_text is empty
empty_og_text_reports = cxr_reports[cxr_reports['cxr_report_text'].isna() | (cxr_reports['cxr_report_text'] == '')]

# Display or print the results
print("Number of reports with empty original text:", len(empty_og_text_reports))
empty_og_text_reports


Number of reports with empty original text: 0


,subject_id,study_id,cxr_report_text,processed_text


In [27]:
# Perform an inner join on processed_text to get all matching records from both tables
merged_df = mimic_only_notes.merge(cxr_reports, on='processed_text', suffixes=('_mimic', '_cxr'))

# Display the count of matches and the merged DataFrame
print("Number of matched rows based on processed_text:", len(merged_df))
merged_df


Number of matched rows based on processed_text: 522


,note_id,subject_id_mimic,hadm_id,note_type,note_seq,charttime,storetime,text,processed_text,subject_id_cxr,study_id,cxr_report_text
0,10033312-RR-72,10033312,NaN,RR,72,2156-01-18 13:11:00,2156-01-18 13:40:00,HISTORY: Preoperative.\n\nFINDINGS: In compa...,HISTORYPreoperativeFINDINGSIncomparisonwiththe...,13649383,53860038,WET READ: ___ ___ 12:30 AM\n No acute cardio...
1,10683716-RR-88,10683716,NaN,RR,88,2128-03-03 12:52:00,2128-03-03 14:49:00,HISTORY: Preoperative.\n\nFINDINGS: In compa...,HISTORYPreoperativeFINDINGSIncomparisonwiththe...,13649383,53860038,WET READ: ___ ___ 12:30 AM\n No acute cardio...
2,14302303-RR-73,14302303,NaN,RR,73,2163-08-19 09:33:00,2163-08-19 10:18:00,HISTORY: Preoperative.\n\nFINDINGS: In compa...,HISTORYPreoperativeFINDINGSIncomparisonwiththe...,13649383,53860038,WET READ: ___ ___ 12:30 AM\n No acute cardio...
3,17474458-RR-22,17474458,NaN,RR,22,2169-01-16 08:58:00,2169-01-16 10:10:00,HISTORY: Preoperative.\n\nFINDINGS: In compa...,HISTORYPreoperativeFINDINGSIncomparisonwiththe...,13649383,53860038,WET READ: ___ ___ 12:30 AM\n No acute cardio...
4,17705128-RR-29,17705128,NaN,RR,29,2158-12-27 11:08:00,2158-12-27 11:42:00,HISTORY: Preoperative.\n\nFINDINGS: In compa...,HISTORYPreoperativeFINDINGSIncomparisonwiththe...,13649383,53860038,WET READ: ___ ___ 12:30 AM\n No acute cardio...
...,...,...,...,...,...,...,...,...,...,...,...,...
517,19687331-RR-30,19687331,NaN,RR,30,2137-11-25 17:42:00,2137-11-25 18:05:00,CHEST RADIOGRAPHS\n\nHISTORY: Chest pain.\n\n...,CHESTRADIOGRAPHSHISTORYChestpainCOMPARISONSTEC...,19345406,51915597,FINAL REPORT\...
518,19782796-RR-15,19782796,NaN,RR,15,2135-01-14 11:33:00,2135-01-14 11:42:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...,EXAMINATIONCHEST(PAANDLAT)INDICATIONHistorywit...,15303898,50642475,FINAL REPORT\...
519,19828656-RR-9,19828656,NaN,RR,9,2117-11-29 22:28:00,2117-11-29 22:43:00,EXAMINATION:\nChest: Frontal and lateral view...,EXAMINATIONChestFrontalandlateralviewsINDICATI...,10882911,54188933,FINAL REPORT\...
520,19849776-RR-23,19849776,NaN,RR,23,2170-03-08 17:02:00,2170-03-08 17:14:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...,EXAMINATIONCHEST(PAANDLAT)INDICATIONwithchestp...,13769676,55310897,FINAL REPORT\...


In [32]:
print('MIMIC Note:',merged_df.iloc[0].text)
print('MIMIC-CXR:',merged_df.iloc[0].cxr_report_text)

MIMIC Note: HISTORY:  Preoperative.

FINDINGS:  In comparison with the study of ___, there is little change and
no evidence of acute cardiopulmonary disease.  No pneumonia, vascular
congestion, or pleural effusion.

MIMIC-CXR:  WET READ: ___ ___ 12:30 AM
  No acute cardiopulmonary process. Known T-spine fracture not well seen on this
  single frontal view.  ___ p___WET READ VERSION #1 
 ______________________________________________________________________________
                                 FINAL REPORT
 HISTORY:  Preoperative.
 
 FINDINGS:  In comparison with the study of ___, there is little change and
 no evidence of acute cardiopulmonary disease.  No pneumonia, vascular
 congestion, or pleural effusion.



In [33]:
print('MIMIC Note:',merged_df.iloc[1].text)
print('MIMIC-CXR:',merged_df.iloc[1].cxr_report_text)

MIMIC Note: HISTORY:  Preoperative.

FINDINGS:  In comparison with the study of ___, there is little change and
no evidence of acute cardiopulmonary disease.  No pneumonia, vascular
congestion, or pleural effusion.

MIMIC-CXR:  WET READ: ___ ___ 12:30 AM
  No acute cardiopulmonary process. Known T-spine fracture not well seen on this
  single frontal view.  ___ p___WET READ VERSION #1 
 ______________________________________________________________________________
                                 FINAL REPORT
 HISTORY:  Preoperative.
 
 FINDINGS:  In comparison with the study of ___, there is little change and
 no evidence of acute cardiopulmonary disease.  No pneumonia, vascular
 congestion, or pleural effusion.



In [38]:
print('MIMIC Note:',merged_df.iloc[517].text)
print('MIMIC-CXR:',merged_df.iloc[517].cxr_report_text)

MIMIC Note: CHEST RADIOGRAPHS

HISTORY:  Chest pain.

COMPARISONS:  ___.

TECHNIQUE:  Chest, PA and lateral.

FINDINGS:

The heart is normal in size.  The mediastinal and hilar contours appear within
normal limits.  There is no pleural effusion or pneumothorax.  The lungs
appear clear.

IMPRESSION:  No evidence of acute cardiopulmonary disease.

MIMIC-CXR:                                  FINAL REPORT
 CHEST RADIOGRAPHS
 
 HISTORY:  Chest pain.
 
 COMPARISONS:  ___.
 
 TECHNIQUE:  Chest, PA and lateral.
 
 FINDINGS:    The heart is normal in size.  The mediastinal and hilar contours
 appear within normal limits.  There is no pleural effusion or pneumothorax. 
 The lungs appear clear. 
 
 IMPRESSION:  No evidence of acute cardiopulmonary disease.



In [30]:
merged_df.iloc[0].cxr_report_text

' WET READ: ___ ___ 12:30 AM\n  No acute cardiopulmonary process. Known T-spine fracture not well seen on this\n  single frontal view.  ___ p___WET READ VERSION #1 \n ______________________________________________________________________________\n                                 FINAL REPORT\n HISTORY:  Preoperative.\n \n FINDINGS:  In comparison with the study of ___, there is little change and\n no evidence of acute cardiopulmonary disease.  No pneumonia, vascular\n congestion, or pleural effusion.\n'

In [10]:
common_subject_mimic_notes = mimic_notes[~mimic_notes['subject_id'].isin(mimic_only_subject_ids)]
len(common_subject_mimic_notes)

1097916

In [11]:
print(len(common_subject_mimic_notes)+len(mimic_only_notes)==len(mimic_notes))

True


In [12]:
len(mimic_only_notes)

1223439

In [13]:
common_subject_mimic_notes['processed_text'] = common_subject_mimic_notes['text'].progress_apply(preprocess_mimic_notes)

100%|██████████| 1097916/1097916 [04:44<00:00, 3853.74it/s]
/scratch/baj321/conda-envs/medfuse_py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
common_subject_cxr_reports = cxr_reports[~cxr_reports['subject_id'].isin(cxr_only_subject_ids)]
len(common_subject_cxr_reports)

215822

In [15]:
common_subject_cxr_reports['processed_text'] = common_subject_cxr_reports['cxr_report_text'].progress_apply(preprocess)

100%|██████████| 215822/215822 [00:29<00:00, 7288.36it/s]
/scratch/baj321/conda-envs/medfuse_py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# Merge the two dataframes on 'subject_id'
merged_common_df = common_subject_mimic_notes.merge(common_subject_cxr_reports, on='subject_id', suffixes=('_mimic', '_cxr'))
# Filter to retain rows where processed texts are different
#merged_common_df = merged_common_df[merged_common_df['processed_text_mimic'] != merged_common_df['processed_text_cxr']]

# Define a function to calculate Jaccard similarity between two sets of text
def jaccard_similarity(text1, text2):
    set1 = set(text1.split())
    set2 = set(text2.split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Define a function that processes a single subject_id group
def process_subject_group(group):
    mimic_entries = group['text'].unique()
    cxr_entries = group['cxr_report_text'].unique()
    
    keep_rows = []

    for idx, row in group.iterrows():
        mimic_text = row['text']
        cxr_text = row['cxr_report_text']
        
        # Check for similarity only between mimic_text and cxr_text entries
        mimic_similar = any(jaccard_similarity(mimic_text, cxr) > 0.9 for cxr in cxr_entries)
        cxr_similar = any(jaccard_similarity(cxr_text, mimic) > 0.9 for mimic in mimic_entries)
        
        # Keep the row only if mimic_text is not too similar to any cxr_text, and vice versa
        if not mimic_similar and not cxr_similar:
            keep_rows.append(row)

    # Return the filtered rows for this group
    return pd.DataFrame(keep_rows)


# Split the DataFrame by subject_id
grouped = list(merged_common_df.groupby('subject_id'))

# Use the number of available CPUs or a set number if desired
num_cores = min(cpu_count(), 28)

with Pool(num_cores) as pool:
    # Process each group in parallel and combine the results
    results = list(tqdm(pool.imap(process_subject_group, [group for _, group in grouped]), total=len(grouped), desc="Filtering by Subject ID"))
    
# Concatenate the filtered results into a single DataFrame
filtered_df = pd.concat(results, ignore_index=True)
    
# `filtered_df` now contains the filtered data, ensuring mimic_texts are only compared to cxr_texts and vice versa


Filtering by Subject ID: 100%|██████████| 61717/61717 [24:23<00:00, 42.17it/s]  


In [17]:
len(filtered_df)

560536

In [18]:
filtered_df.columns

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text', 'processed_text_mimic', 'study_id',
       'cxr_report_text', 'processed_text_cxr'],
      dtype='object')

In [19]:
# Columns related to cxr
cxr_columns = ['subject_id','study_id', 'cxr_report_text', 'processed_text_cxr']

# Columns related to mimic
mimic_columns = ['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq', 'charttime', 'storetime', 'text', 'processed_text_mimic' ]

# Creating two new DataFrames
cxr_df = filtered_df[cxr_columns].drop_duplicates()
mimic_df = filtered_df[mimic_columns].drop_duplicates()


In [26]:
len(cxr_df)

36717

In [27]:
len(mimic_df)

470964

In [24]:
mimic_notes.columns

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text'],
      dtype='object')

In [28]:
filtered_df.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,processed_text_mimic,study_id,cxr_report_text,processed_text_cxr
0,10000980-RR-52,10000980,NaN,RR,52,2185-06-27 14:21:00,2185-06-28 12:16:00,HISTORY: Screening.\n\nDIGITAL SCREENING MAMM...,HISTORYScreeningDIGITALSCREENINGMAMMOGRAPHYINT...,58206436,WET READ: ___ ___ ___ 6:47 AM\n 1. New mild ...,EXAMINATIONChestradiographINDICATIONwithwheezi...
1,10000980-RR-55,10000980,NaN,RR,55,2187-01-10 09:16:00,2187-01-13 16:47:00,INDICATION: Screening. \n\nDIGITAL SCREENING ...,INDICATIONScreeningDIGITALSCREENINGMAMMOGRAMWI...,58206436,WET READ: ___ ___ ___ 6:47 AM\n 1. New mild ...,EXAMINATIONChestradiographINDICATIONwithwheezi...
2,10000980-RR-56,10000980,NaN,RR,56,2187-02-21 04:11:00,2187-02-21 06:01:00,HISTORY: ___ female with near syncopal episod...,HISTORYfemalewithnearsyncopalepisodeSTUDYPAand...,58206436,WET READ: ___ ___ ___ 6:47 AM\n 1. New mild ...,EXAMINATIONChestradiographINDICATIONwithwheezi...
3,10000980-RR-57,10000980,NaN,RR,57,2187-02-21 06:38:00,2187-02-21 07:34:00,"HISTORY: ___ female with prior CVA, now prese...",HISTORYfemalewithpriorCVAnowpresentingwithTIAs...,58206436,WET READ: ___ ___ ___ 6:47 AM\n 1. New mild ...,EXAMINATIONChestradiographINDICATIONwithwheezi...
4,10000980-RR-60,10000980,NaN,RR,60,2189-01-26 10:32:00,2189-02-01 12:45:00,HISTORY: Screening. \n\nDIGITAL HOLOGIC SCRE...,HISTORYScreeningDIGITALHOLOGICSCREENINGMAMMOGR...,58206436,WET READ: ___ ___ ___ 6:47 AM\n 1. New mild ...,EXAMINATIONChestradiographINDICATIONwithwheezi...


In [ ]:
from tqdm import tqdm

# Initialize tqdm for progress tracking
tqdm.pandas()

# Filter `common_subject_mimic_notes` to keep rows where there's no match in `common_subject_cxr_reports`
filtered_mimic_notes = common_subject_mimic_notes[
    ~common_subject_mimic_notes.progress_apply(
        lambda row: ((common_subject_cxr_reports['subject_id'] == row['subject_id']) &
                     (common_subject_cxr_reports['processed_text'] == row['processed_text'])).any(),
        axis=1
    )
]

# Filter `common_subject_cxr_reports` to keep rows where there's no match in `common_subject_mimic_notes`
filtered_cxr_reports = common_subject_cxr_reports[
    ~common_subject_cxr_reports.progress_apply(
        lambda row: ((common_subject_mimic_notes['subject_id'] == row['subject_id']) &
                     (common_subject_mimic_notes['processed_text'] == row['processed_text'])).any(),
        axis=1
    )
]
